In [1]:
import tensorflow as tf
import numpy as np

In [2]:
train_file = open("../../data/iyer/train.txt")

In [3]:
train_file_contents = train_file.readlines()

In [4]:
len(train_file_contents)

52997

In [5]:
train = []
for line in train_file_contents:
    items = line.split('\t')
    if len(items) == 5:
        train.append(line.split('\t')[2])

In [6]:
len(train)

52795

In [7]:
len(train[0])

31

In [8]:
len(train[1])

23

In [10]:
import sys
sys.path.append("../../src")
from text_data_utils import *
from os import path
import gensim

In [11]:
train_tok = tokenize_texts(train)

In [12]:
wv = gensim.models.Word2Vec(train_tok).wv

In [13]:
max_len = max(len(text) for text in train_tok)

In [30]:
train_tensor = tokenized_texts_to_tensor(train_tok, wv, max_len)

In [31]:
train_tensor_fl = np.reshape(train_tensor, (train_tensor.shape[0], train_tensor.shape[1] * train_tensor.shape[2]))

In [32]:
train_tensor.shape

(52795, 39, 100)

In [33]:
train_tensor_fl.shape

(52795, 3900)

In [34]:
val_file_contents = open('../../data/iyer/valid.txt').readlines()
val = []
for line in val_file_contents:
    items = line.split('\t')
    if len(items) == 5:
        val.append(line.split('\t')[2])
val_tok = tokenize_texts(val)
val_tensor = tokenized_texts_to_tensor(val_tok, wv, max_len)
val_tensor_fl = np.reshape(val_tensor, (val_tensor.shape[0], val_tensor.shape[1] * val_tensor.shape[2]))

In [35]:
from mlp_vae import MLPVariationalAutoEncoder

In [36]:
latent_dim = 128

In [38]:
model = MLPVariationalAutoEncoder(train_tensor_fl.shape[1], latent_dim, [1024, 512])
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(train_tensor_fl, train_tensor_fl, batch_size=256, epochs=12, verbose=1, shuffle=True,
                    validation_data=(val_tensor_fl, val_tensor_fl))

Train on 52795 samples, validate on 6599 samples
Epoch 1/12
52795/52795 [==============================] - 8s 154us/sample - loss: 0.1248 - val_loss: 0.1086
Epoch 2/12
52795/52795 [==============================] - 7s 127us/sample - loss: 0.1057 - val_loss: 0.1063
Epoch 3/12
52795/52795 [==============================] - 7s 127us/sample - loss: 0.1018 - val_loss: 0.1019
Epoch 4/12
52795/52795 [==============================] - 7s 129us/sample - loss: 0.0988 - val_loss: 0.0974
Epoch 5/12
52795/52795 [==============================] - 7s 125us/sample - loss: 0.0960 - val_loss: 0.0962
Epoch 6/12
52795/52795 [==============================] - 7s 126us/sample - loss: 0.0938 - val_loss: 0.0926
Epoch 7/12
52795/52795 [==============================] - 7s 126us/sample - loss: 0.0922 - val_loss: 0.0931
Epoch 8/12
52795/52795 [==============================] - 7s 127us/sample - loss: 0.0915 - val_loss: 0.0920
Epoch 9/12
52795/52795 [==============================] - 7s 125us/sample - loss: 0.090

In [40]:
print("(Training Set) Input: ", tensor_to_tokenized_texts(np.array([train_tensor[0]]), wv)[0])

(Training Set) Input:  ['<s>', 'C', '#', '<UNK>', ',', 'setters', 'declaration', '</s>']


In [43]:
rec = np.reshape(model.predict(np.array([train_tensor_fl[0]])), (1, train_tensor.shape[1], train_tensor.shape[2]))

In [44]:
print("(Training Set) Reconstructed: ", tensor_to_tokenized_texts(rec, wv)[0])

(Training Set) Reconstructed:  ['<s>', 'C', '#', 'Datagridview', ',', 'Not', 'inserted', '</s>']


In [46]:
wv.similar_by_vector(rec[0, 5])

[('Not', 0.6574020385742188),
 ('direct', 0.6554468870162964),
 ('Customizing', 0.6533945202827454),
 ('Frame', 0.6525598764419556),
 ('Only', 0.6509501934051514),
 ('xmlns', 0.6459565162658691),
 ('tracking', 0.6429063081741333),
 ('trust', 0.6414172649383545),
 ('Notification', 0.6399063467979431),
 ('support', 0.639179527759552)]

In [47]:
test_file_contents = open('../../data/iyer/test.txt').readlines()
test = []
for line in test_file_contents:
    items = line.split('\t')
    if len(items) == 5:
        test.append(line.split('\t')[2])
test_tok = tokenize_texts(test)
test_tensor = tokenized_texts_to_tensor(test_tok, wv, max_len)
test_tensor_fl = np.reshape(test_tensor, (test_tensor.shape[0], test_tensor.shape[1] * test_tensor.shape[2]))

In [49]:
model.evaluate(test_tensor_fl, test_tensor_fl, verbose=0)

0.11597092485568762

In [50]:
import random

In [53]:
random_idx = random.randrange(test_tensor.shape[0])

In [57]:
print("(Test Set) Input: ", tensor_to_tokenized_texts(np.array([test_tensor[random_idx]]), wv)[0])

(Test Set) Input:  ['<s>', 'Using', 'LINQ', 'to', 'select', 'a', 'random', 'XML', 'node', '</s>']


In [60]:
print("(Test Set) Reconstructed: ", tensor_to_tokenized_texts(np.reshape(model.predict(np.array([test_tensor_fl[random_idx]])), (1, test_tensor.shape[1], test_tensor.shape[2])), wv)[0])

(Test Set) Reconstructed:  ['<s>', 'Join', 'Group', 'to', 'remove', 'a', 'contents', 'excel', 'contents', '</s>']
